In [2]:
import numpy as np
import cv2
from Bio.PDB import PDBParser

In [3]:
def extract_water_coords(pdb_path):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("water", pdb_path)
    coords = []

    for model in structure:
        for chain in model:
            for residue in chain:
                for atom in residue:
                    coords.append(atom.coord)
    return np.array(coords)

In [4]:
def project_to_2d(coords_3d, img_size=(512, 512), axis='z'):
    if axis == 'z':
        projected = coords_3d[:, :2]
    elif axis == 'y':
        projected = coords_3d[:, [0, 2]]
    elif axis == 'x':
        projected = coords_3d[:, 1:]
    else:
        raise ValueError("Invalid axis")

    # Normalize to image pixel space
    min_vals = projected.min(axis=0)
    max_vals = projected.max(axis=0)
    norm_coords = (projected - min_vals) / (max_vals - min_vals + 1e-8)
    pixel_coords = (norm_coords * np.array(img_size)).astype(int)
    return pixel_coords

In [5]:
def create_mask(pixel_coords, img_size=(512, 512), radius=3):
    mask = np.zeros(img_size, dtype=np.uint8)
    for (x, y) in pixel_coords:
        if 0 <= x < img_size[0] and 0 <= y < img_size[1]:
            cv2.circle(mask, (x, y), radius, 255, -1)  # white circles
    return mask

In [7]:
inpath = "/Users/yeonji/Dropbox/myfolder_data/wbp_last/ahr_eq/cc/"
outpath = "/Users/yeonji/Desktop/ComputerProject/CNN_HS_Predict/hs_image/"
coords = extract_water_coords(inpath + "xiap_ahr_eq_cc.pdb")
pixel_coords = project_to_2d(coords, img_size=(512, 512), axis='z')
mask = create_mask(pixel_coords, img_size=(512, 512), radius=4)
cv2.imwrite(outpath + "xiap_ahr.png", mask)

True

In [9]:
import glob
hs_files = glob.glob(inpath + "*pdb")
for file in hs_files:
    target = file.split("/")[-1].split("_")[0]
    coords = extract_water_coords(file)
    pixel_coords = project_to_2d(coords, img_size=(512, 512), axis='z')
    mask = create_mask(pixel_coords, img_size=(512, 512), radius=4)
    cv2.imwrite(outpath + target + "_ahr.png", mask)